In [1]:
import numpy as np
import scanorama
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
import time
from datetime import timedelta

scanpy==1.4.1 anndata==0.6.19 numpy==1.14.3 scipy==1.1.0 pandas==0.24.2 scikit-learn==0.20.0 statsmodels==0.8.0 python-igraph==0.7.1 louvain==0.6.1 


In [2]:
import os 
dirname = os.getcwd()
print(dirname)

data_dir = '/acrc/jinmiao/CJM_lab/hoatran/demo_normalization/dataset/dataset10_hematoMNN_Hoa/final/'

/acrc/jinmiao/CJM_lab/Marion/Project/Hoa_batch_normalization/demo_scanorama/scanorama_dataset10


In [3]:
# Create folder to save the results 
#if not os.path.exists('./scanorama_dataset4/'): os.makedirs('./scanorama_dataset4/')
    
save_dir = dirname
def save_images(basename):
    if not os.path.exists('./visualization'): os.makedirs('./visualization')
    
    outname = './visualization/' + basename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

In [4]:
# read data from read count text table, data from R: genes x cells
adata1 = pd.read_csv(os.path.join(data_dir,'b1_exprs.txt'),sep='\t',header=0, index_col=0)
adata2 = pd.read_csv(os.path.join(data_dir,'b2_exprs.txt'),sep='\t',header=0, index_col=0)
adata = pd.concat([adata1, adata2], axis=1)
adata = sc.AnnData(np.transpose(adata))
print(adata)
print(adata.obs_names[1:3])
print(adata.var_names[1:3])

AnnData object with n_obs × n_vars = 4649 × 3467 
Index(['W31107', 'W31108'], dtype='object')
Index(['ENSMUSG00000000290', 'ENSMUSG00000000594'], dtype='object')


In [5]:
sample_adata1 = pd.read_csv(os.path.join(data_dir,'b1_celltype.txt'),header=0, index_col=0, sep='\t')
sample_adata1['batch'] = 'batch1'
sample_adata2 = pd.read_csv(os.path.join(data_dir,'b2_celltype.txt'),header=0, index_col=0, sep='\t')
sample_adata2['batch'] = 'batch2'
sample_adata = pd.concat([sample_adata1, sample_adata2], axis=0)
print(sample_adata.values.shape)
print(sample_adata.keys())

(4649, 2)
Index(['CellType', 'batch'], dtype='object')


In [7]:
# adata.obs['cell_type'] = sample_adata.loc[adata.obs_names,['CellType']]
# adata.obs['batch'] = sample_adata.loc[adata.obs_names,['batch']]
# adata.obs['tissue'] = sample_adata.loc[adata.obs_names,['tissue']]
# adata.obs['organ'] = sample_adata.loc[adata.obs_names,['organ']]
# adata.obs['batchlb'] = sample_adata.loc[adata.obs_names,['batchlb']]

# adata.write(os.path.join(save_dir,'dataset10.h5ad'))

# Save output into h5ad, easy to access 
adata1 = sc.read_h5ad(os.path.join(save_dir,'dataset10.h5ad'))
adata1

... storing 'cell_type' as categorical
... storing 'batch' as categorical


AnnData object with n_obs × n_vars = 4649 × 3467 
    obs: 'cell_type', 'batch'

In [8]:
from scanorama import correct, visualize, process_data
from scanorama import dimensionality_reduce

In [9]:
adata_b1 = adata1[adata1.obs['batch']=='batch1',:].copy()
print(adata_b1)
adata_b2 = adata1[adata1.obs['batch']=='batch2',:].copy()
print(adata_b2)

AnnData object with n_obs × n_vars = 2729 × 3467 
    obs: 'cell_type', 'batch'
AnnData object with n_obs × n_vars = 1920 × 3467 
    obs: 'cell_type', 'batch'


In [10]:
adata_ls = [adata_b1, adata_b2]

In [11]:
# Batch correction.
# datasets = [df.values]
# List of datasets (matrices of cells-by-genes)
t1 = time.time()
corrected = scanorama.correct_scanpy(adata_ls, batch_size=50, return_dense=True, knn=20)
# corrected, genes = scanorama.correct(adata_ls, adata1.var_names,batch_size=30)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

Found 3467 genes among all datasets
[[0.         0.32083333]
 [0.         0.        ]]
Processing datasets (0, 1)
Took 0:00:19.962375


In [12]:
total_ann = sc.AnnData(np.concatenate([corrected[0].X, corrected[1].X]))
print(total_ann)

AnnData object with n_obs × n_vars = 4649 × 3467 


In [13]:
total_ann.var_names = corrected[0].var_names
total_ann.obs_names = corrected[0].obs_names.tolist() + corrected[1].obs_names.tolist()
total_ann.obs['cell_type'] = corrected[0].obs['cell_type'].tolist() + corrected[1].obs['cell_type'].tolist()
total_ann.obs['batch'] = corrected[0].obs['batch'].tolist() + corrected[1].obs['batch'].tolist()
total_ann

AnnData object with n_obs × n_vars = 4649 × 3467 
    obs: 'cell_type', 'batch'

In [14]:
sc.tl.pca(total_ann, svd_solver='arpack') # n_comps=20
print('before: ',total_ann.obsm['X_pca'][1:2])
total_ann.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
print('after multiply: ',total_ann.obsm['X_pca'][1:2])

before:  [[-4.98238117e-01 -2.74382323e-01  1.07898876e-01 -1.47574325e-03
  -2.28505768e-02 -1.01060539e-01 -2.88288873e-02  2.80157104e-02
  -2.69658528e-02  2.46563889e-02  4.58510257e-02 -4.00498658e-02
   3.53493914e-02  6.75095199e-03 -9.03615430e-02 -3.12630013e-02
   1.82413694e-03  8.13910440e-02 -1.11637823e-02 -2.06006486e-02
   8.11845344e-03  4.77864221e-03  1.87898353e-02 -2.19625738e-02
  -5.64173050e-02 -7.82674551e-03 -2.21576579e-02 -7.18864053e-03
  -3.70230875e-03  2.23114789e-02 -3.84375424e-04 -2.61909459e-02
  -2.79475078e-02  6.62431261e-03  1.76506378e-02 -2.29860283e-02
  -2.95176618e-02  1.02725960e-02  5.78616373e-03  1.79089829e-02
   1.05333636e-02 -3.93981626e-03  1.87920965e-02  2.83901989e-02
  -3.75365727e-02 -2.45604161e-02  3.13864015e-02 -1.42551083e-02
   4.39827964e-02  2.86660399e-02]]
after multiply:  [[ 4.98238117e-01  2.74382323e-01 -1.07898876e-01  1.47574325e-03
   2.28505768e-02  1.01060539e-01  2.88288873e-02 -2.80157104e-02
   2.69658528e

In [15]:
total_ann.write_h5ad(os.path.join(save_dir,'scanorama_corrected.h5ad'))

... storing 'cell_type' as categorical
... storing 'batch' as categorical


In [16]:
# save run time
if not os.path.exists(os.path.join(save_dir,'runtime/')): os.makedirs(os.path.join(save_dir,'runtime/'))

print(t1)
print(t2)
time_taken = t2 - t1
time_taken_mins = divmod(time_taken, 60)
time_taken_hours, rest = divmod( time_taken, 3600)
hours_mins, hours_secs = divmod( rest, 60)
print('Took seconds: '+str(timedelta(seconds=round(time_taken))))
print('Took minutes: '+str(time_taken_mins))
print('Took hours_minutes_seconds: ',str(time_taken_hours),str(hours_mins),str(hours_secs))
usecase_name = 'scanorama'
filename = 'scanorama_runtime.txt'

data = {'use_case':usecase_name, 'exetime_secs':str(round(time_taken)),
       'exetimehours': str(time_taken_hours),
       'exetimemins': str(hours_mins),
       'exetimesecs':str(round(hours_secs))} 
  
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['exetime'])
print(df)
print("Save file into directory: ",save_dir)
df.to_csv(os.path.join(save_dir,'runtime/',filename)) 

1563182762.8654664
1563182782.8278415
Took seconds: 0:00:20
Took minutes: (0.0, 19.962375164031982)
Took hours_minutes_seconds:  0.0 0.0 19.962375164031982
          use_case exetime_secs exetimehours exetimemins exetimesecs
exetime  scanorama           20          0.0         0.0          20
Save file into directory:  /acrc/jinmiao/CJM_lab/Marion/Project/Hoa_batch_normalization/demo_scanorama/scanorama_dataset10


In [10]:
# Function to plot TSNE
def plotTSNE(adata, color_group, n_pcs=20, perplexity=90, save_filename='tsne', use_repx = False):
    #adata.var_names_make_unique()
    if use_repx:
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity, use_rep='X')
    else:    
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename)     
    
def plotUMAP(adata, color_group, save_filename, use_repx = False):
    
#     if use_repx:
#         sc.pp.neighbors(adata, use_rep='X')
#     else:    
#         sc.pp.neighbors(adata,n_neighbors=10, n_pcs=20)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename) 

In [11]:
# Visulization of normalized data
# sc.tl.pca(total_ann, svd_solver='arpack')
sc.pp.neighbors(total_ann,n_neighbors=15, n_pcs=20)
print('scanorama - batch effect correction')
color_group = ['cell_type','batch']
# Visualization UMAP
plotUMAP(total_ann, color_group, 'scanorama_umap')
# Visualization tSNE
plotTSNE(total_ann, color_group, 10, 30, 'scanorama_tsne')

computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:05:43.35) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
scanorama - batch effect correction
computing UMAP
    finished (0:00:28.05) --> added
    'X_umap', UMAP coordinates (adata.obsm)
computing tSNE
    using 'X_pca' with n_pcs = 10
    using sklearn.manifold.TSNE with a fix by D. DeTomaso
    finished (0:00:49.84) --> added
    'X_tsne', tSNE coordinates (adata.obsm)


In [12]:
# scanorama_ann.obsm['X_umap'].__class__
colnu = []
for i in range(total_ann.obsm['X_umap'].shape[1]):
    colnu.append("UMAP"+str(i+1))
df = pd.DataFrame(total_ann.obsm['X_umap'], columns=colnu, index=total_ann.obs_names)
df['batch'] = pd.Series(total_ann.obs['batch'], index=total_ann.obs_names)
df['celltype'] = pd.Series(total_ann.obs['cell_type'], index=total_ann.obs_names)
df.to_csv(save_dir + '/visualization/scanorama_umap.csv') 

In [13]:
colnt = []
for i in range(total_ann.obsm['X_tsne'].shape[1]):
    colnt.append("tSNE_"+str(i+1))
df = pd.DataFrame(total_ann.obsm['X_tsne'], columns=colnt, index=total_ann.obs_names)
df['batch'] = pd.Series(total_ann.obs['batch'], index=total_ann.obs_names)
df['celltype'] = pd.Series(total_ann.obs['cell_type'], index=total_ann.obs_names)
df.to_csv(save_dir+'/visualization/scanorama_tsne.csv') 

In [14]:
# save
if not os.path.exists(os.path.join(save_dir,'ASW/')): os.makedirs(os.path.join(save_dir,'ASW/'))

total_ann.obsm['X_pca'].shape
print(total_ann.obsm['X_pca'][1:5])
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+'/ASW/'+filename)  
    
filename = 'scanorama_pca.csv'
coln_pca = []
for i in range(total_ann.obsm['X_pca'].shape[1]):
    coln_pca.append("X_pca"+str(i+1))
    
write_to_csv(total_ann.obsm['X_pca'], coln_pca, total_ann.obs_names, filename, save_dir)

[[ 4.98238117e-01  2.74382323e-01 -1.07898876e-01  1.47574325e-03
   2.28505768e-02  1.01060539e-01  2.88288873e-02 -2.80157104e-02
   2.69658528e-02 -2.46563889e-02 -4.58510257e-02  4.00498658e-02
  -3.53493914e-02 -6.75095199e-03  9.03615430e-02  3.12630013e-02
  -1.82413694e-03 -8.13910440e-02  1.11637823e-02  2.06006486e-02
  -8.11845344e-03 -4.77864221e-03 -1.87898353e-02  2.19625738e-02
   5.64173050e-02  7.82674551e-03  2.21576579e-02  7.18864053e-03
   3.70230875e-03 -2.23114789e-02  3.84375424e-04  2.61909459e-02
   2.79475078e-02 -6.62431261e-03 -1.76506378e-02  2.29860283e-02
   2.95176618e-02 -1.02725960e-02 -5.78616373e-03 -1.79089829e-02
  -1.05333636e-02  3.93981626e-03 -1.87920965e-02 -2.83901989e-02
   3.75365727e-02  2.45604161e-02 -3.13864015e-02  1.42551083e-02
  -4.39827964e-02 -2.86660399e-02]
 [-5.21412671e-01  2.65941322e-01 -1.40245371e-02  3.21648382e-02
   1.98735390e-02 -3.13851349e-02  2.35952772e-02  2.64317822e-02
   1.00202076e-01  4.89457063e-02 -1.9641